In [1]:
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from neurals.network import PCN, PointNet2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


pybullet build time: May 20 2022 19:44:17


In [2]:
pcds = []
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_0_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_1_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_2_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_3_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_4_pcd.ply"))
pcds.append(o3d.io.read_point_cloud("data/seeds/pointclouds/pose_5_pcd.ply"))

In [3]:
points_list = []
for pcd in pcds:
    for i in range(100):
        points = np.asarray(pcd.points)
        points_index = np.random.choice(len(points), 1024, replace=False)
        points = points[points_index]
        points_list.append(points)
print(len(points_list))

600


In [4]:
class TestDataset(Dataset):
    def __init__(self, points_list):
        self.pcds = np.asarray(points_list)
        self.mapping = []
        for i in range(6):
            self.mapping += [i] * 100
        self.mapping = np.array(self.mapping).astype(np.int64)

    def __len__(self):
        return len(self.mapping)

    def __getitem__(self, index):
        mapping = self.mapping[index]
        pcd = self.pcds[index]
        ret_pcd = pcd + np.random.normal(size=pcd.shape, scale = 0.005)
        return mapping, ret_pcd

dataset = TestDataset(points_list)
(training_dataset, test_dataset) = \
        torch.utils.data.random_split(
            dataset, (550, 50))

In [5]:
class TestNN(nn.Module):
    def __init__(self):
        super(TestNN, self).__init__()
        self.pcn = PCN(latent_dim=10)
        #self.pcn = PointNet2(feat_dim=0)
        self.fc1 = nn.Linear(10, 64)
        self.out = nn.Linear(64, 6)

    def forward(self, pcd):
        latent = self.pcn(pcd)
        x = self.fc1(latent).relu()
        x = torch.nn.functional.log_softmax(self.out(x), dim=1)
        return x
net = TestNN().cuda()

In [6]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = torch.nn.NLLLoss()
dataloader = DataLoader(training_dataset, batch_size=64)

In [7]:
for i in range(500):
    batch_cnt = 0
    total_loss = 0
    for label, pointcloud in dataloader:
        optim.zero_grad()
        label = label.cuda()
        pointcloud = pointcloud.cuda().float()
        pred = net(pointcloud)
        loss = loss_fn(pred, label)
        loss.backward()
        optim.step()
        batch_cnt += 1
        total_loss += float(loss)
    print(f"Total Loss: {total_loss/batch_cnt}")

Total Loss: 1.7964445352554321
Total Loss: 1.7928923765818279
Total Loss: 1.7919674846861098
Total Loss: 1.791657222641839
Total Loss: 1.791511336962382
Total Loss: 1.7914344469706218
Total Loss: 1.7913611200120714
Total Loss: 1.7913244432873197
Total Loss: 1.79126328892178
Total Loss: 1.791198558277554
Total Loss: 1.7911457353168063
Total Loss: 1.7910645008087158
Total Loss: 1.7909821536805894
Total Loss: 1.790849208831787
Total Loss: 1.7907042900721233
Total Loss: 1.7905460596084595
Total Loss: 1.7902708583407931
Total Loss: 1.7898679706785414
Total Loss: 1.7894263797336154
Total Loss: 1.7886549234390259
Total Loss: 1.7875872453053792
Total Loss: 1.7856053908665974
Total Loss: 1.7821884287728205
Total Loss: 1.7769216696421306
Total Loss: 1.7686369684007432
Total Loss: 1.7500575648413763
Total Loss: 1.7258691390355427
Total Loss: 1.6791015598509047
Total Loss: 1.617857853571574
Total Loss: 1.5742187632454767
Total Loss: 1.4961806535720825
Total Loss: 1.3926108015908136
Total Loss: 1.2

In [8]:
dataloader = DataLoader(test_dataset, batch_size=1)
batch_cnt = 0
total_loss = 0
net.eval()
labels = []
for label, pointcloud in dataloader:
    label = label.cuda()
    pointcloud = pointcloud.cuda().float()
    pred = net(pointcloud)
    labels.append(label)
    loss = loss_fn(pred, label)
    batch_cnt += 1
    total_loss += float(loss)
print(f"Total Test Loss: {total_loss/batch_cnt}")

Total Test Loss: 0.001626654759324424


In [9]:
torch.save(net.pcn.state_dict(), "neurals/pcn_model/pcn_10.pth")

In [19]:
o3d.visualization.draw_geometries([o3d.geometry.PointCloud(o3d.utility.Vector3dVector(training_dataset[10][1]))])

In [15]:
test_dataset[10][1]

array([[-0.04219856, -0.01532705,  0.03520654],
       [-0.03941568,  0.10910279, -0.05009749],
       [ 0.03348964, -0.15582641,  0.04298963],
       ...,
       [-0.20387647, -0.08532091,  0.06352441],
       [-0.06860191,  0.14903795,  0.05420021],
       [ 0.10703852, -0.02935053,  0.06121993]])